# World commodity prices

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
from datawrapper import Datawrapper
import json

In [3]:
token = !echo $dw_api
dw = Datawrapper(access_token=token[0])

In [4]:
token

['FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z']

In [5]:
cutoff_date = "2021-09-01"

---

### Wheat

In [6]:
# https://www.wsj.com/market-data/quotes/futures/W00/advanced-chart
# Wheat continuous contract

In [7]:
import requests

headers = {
    "Connection": "keep-alive",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Dylan2010.EntitlementToken": "57494d5ed7ad44af85bc59a51dd87c90",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
    "Origin": "https://www.wsj.com",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Dest": "empty",
    "Referer": "https://www.wsj.com/",
    "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
}

params = {
    "json": '{"Step":"P1D","TimeFrame":"P1Y","StartDate":1616457600000,"EndDate":1647993600000,"EntitlementToken":"57494d5ed7ad44af85bc59a51dd87c90","IncludeMockTick":true,"FilterNullSlots":false,"FilterClosedPoints":true,"IncludeClosedSlots":false,"IncludeOfficialClose":true,"InjectOpen":false,"ShowPreMarket":false,"ShowAfterHours":false,"UseExtendedTimeFrame":true,"WantPriorClose":false,"IncludeCurrentQuotes":false,"ResetTodaysAfterHoursPercentChange":false,"Series":[{"Key":"FUTURE/US/XCBT/W00","Dialect":"Charting","Kind":"Ticker","SeriesId":"s1","DataTypes":["Last"],"Indicators":[{"Parameters":[{"Name":"Period","Value":"200"}],"Kind":"SimpleMovingAverage","SeriesId":"i2"}]}]}',
    "ckey": "57494d5ed7",
}

response = requests.get(
    "https://api.wsj.net/api/michelangelo/timeseries/history",
    headers=headers,
    params=params,
)

#### Extract the features we need

In [8]:
wheat_dates = pd.Series(response.json()["TimeInfo"]["Ticks"])

In [9]:
wheat_values = []

for price in response.json()["Series"][0]["DataPoints"]:
    wheat_values.append(price[0])

In [10]:
wheat_values_series = pd.Series(wheat_values)

In [11]:
wheat_df = pd.concat([wheat_dates, wheat_values_series], axis=1)

In [12]:
wheat_df.columns = ["timestamp", "value"]

In [13]:
wheat_df["date"] = pd.to_datetime(wheat_df["timestamp"], unit="ms").dt.strftime(
    "%Y-%m-%d"
)

In [14]:
wheat_df["dollars_per_bushel"] = (wheat_df["value"] / 100).round(2)

#### Export

In [15]:
wheat_df[wheat_df["date"] > cutoff_date].to_csv(
    "data/processed/tracker_wheat_prices.csv", index=False
)

In [16]:
dw.add_data(chart_id="nHFIW", data=wheat_df[wheat_df["date"] > cutoff_date])

---

### Fertilizer

In [17]:
# Green Markets Weekly North America Fertilizer Price Index Notation
# https://fertilizerpricing.com/priceindex/

In [18]:
fertilizer_raw = pd.read_json(
    "https://fertilizerpricing.com/wp-content/themes/greenmarkets/fcharts/fchart_lib/json/data_open.php"
)

#### Get historical data into a dataframe

In [19]:
historical = []

for d in fertilizer_raw["data"][0]:
    data_dict = {"timestamp": d[0], "value": d[1]}
    historical.append(data_dict)

fertilizer_df = pd.DataFrame(historical)

#### Dates

In [20]:
fertilizer_df["date"] = pd.to_datetime(
    fertilizer_df["timestamp"], unit="ms"
).dt.strftime("%Y-%m-%d")

#### Export

In [21]:
fertilizer_df[fertilizer_df["date"] > cutoff_date].to_csv(
    "data/processed/tracker_fertilizer_prices.csv", index=False
)

In [22]:
dw.add_data(chart_id="WPSUZ", data=fertilizer_df[fertilizer_df["date"] > cutoff_date])

<Response [204]>

---

### Nickel

In [23]:
nickel_src = pd.read_csv("data/raw/nickel.csv")

In [24]:
nickel_src.columns = nickel_src.columns.str.lower()

In [25]:
nickel_src["date"] = pd.to_datetime(nickel_src["date"])

In [26]:
nickel_df = nickel_src[["date", "close"]]

In [27]:
nickel_df[nickel_df["date"] > cutoff_date].to_csv(
    "data/processed/tracker_nickel_prices.csv", index=False
)

In [28]:
dw.add_data(chart_id="1daCt", data=nickel_df[nickel_df["date"] > cutoff_date])

<Response [204]>